# Import Libraries

In [144]:
''' import relevant libaries for manipulating data'''
import requests
import pandas as pd
import pycountry
import copy
import numpy as np

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)

import coinoxr
from coinoxr import Historical
from coinoxr import Currency
from coinoxr.requestor import Requestor

coinoxr.app_id = "114d34f7c3164eacba5c42442ae8c4c5"

from tqdm import tqdm

# Get All Currencies Names and Codes

In [154]:
cur = Currency().get()
currencies = dict(cur.body)
cur

Response(code=200, body={'AED': 'United Arab Emirates Dirham', 'AFN': 'Afghan Afghani', 'ALL': 'Albanian Lek', 'AMD': 'Armenian Dram', 'ANG': 'Netherlands Antillean Guilder', 'AOA': 'Angolan Kwanza', 'ARS': 'Argentine Peso', 'AUD': 'Australian Dollar', 'AWG': 'Aruban Florin', 'AZN': 'Azerbaijani Manat', 'BAM': 'Bosnia-Herzegovina Convertible Mark', 'BBD': 'Barbadian Dollar', 'BDT': 'Bangladeshi Taka', 'BGN': 'Bulgarian Lev', 'BHD': 'Bahraini Dinar', 'BIF': 'Burundian Franc', 'BMD': 'Bermudan Dollar', 'BND': 'Brunei Dollar', 'BOB': 'Bolivian Boliviano', 'BRL': 'Brazilian Real', 'BSD': 'Bahamian Dollar', 'BTC': 'Bitcoin', 'BTN': 'Bhutanese Ngultrum', 'BWP': 'Botswanan Pula', 'BYN': 'Belarusian Ruble', 'BZD': 'Belize Dollar', 'CAD': 'Canadian Dollar', 'CDF': 'Congolese Franc', 'CHF': 'Swiss Franc', 'CLF': 'Chilean Unit of Account (UF)', 'CLP': 'Chilean Peso', 'CNH': 'Chinese Yuan (Offshore)', 'CNY': 'Chinese Yuan', 'COP': 'Colombian Peso', 'CRC': 'Costa Rican Colón', 'CUC': 'Cuban Convert

# Import Nationality and Currency-Nationality Mapping Data
## Remove Currencies without Representation at Minerva

In [155]:
#https://gist.github.com/emmastiefel/c891ff0910a253b1343259c5f27510bd
nationality =  pd.read_csv("data/nationality.csv")
nationality_df = nationality.loc[:, nationality.columns.intersection(['Nationality', 'Total Students'])]


#https://gist.github.com/HarishChaudhari/4680482
mapping =  pd.read_csv("data/map.csv")

c_list = []
for count, co in enumerate(mapping["Country"]):
    if str(co) not in list(nationality_df["Nationality"]):
        c_list.append(count)

mapping = mapping.drop(c_list)

nationality_df = nationality_df.merge(mapping, left_on='Nationality', right_on='Country').drop(columns = ['Country'])

nationality_df = nationality_df.rename(columns={"Code": "Currency Code"})

nationality_df

,Nationality,Total Students,Currency Code
0,Albania,1,ALL
1,Algeria,1,DZD
2,Argentina,3,ARS
3,Armenia,3,AMD
4,Australia,3,AUD
...,...,...,...
72,Uruguay,2,UYU
73,Uzbekistan,1,UZS
74,Venezuela,2,VEF
75,Vietnam,35,VND


## Map Number of Students to Currencies

In [156]:
#multiple countries to same currency, use groupby sum
student_df = nationality_df.groupby(['Currency Code'], as_index=False)['Total Students'].sum()
student_df = student_df.drop(index = 62) #drop venezula, 2 students unaccounted for.
student_df

,Currency Code,Total Students
0,AED,1
1,ALL,1
2,AMD,3
3,ARS,3
4,AUD,3
...,...,...
60,UYU,2
61,UZS,1
63,VND,35
64,ZAR,4


In [157]:
currencies = dict(cur.body)
no_need = list(set(currencies.keys()) - set(student_df['Currency Code']))
for currency in no_need: 
    currencies.pop(currency)
len(currencies)

65

In [265]:
df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
df["Students"] = list(student_df['Total Students'])


df.Students[59] / sum(df.Students)

0.12794612794612795

# Get Historical Forex from API

## By Calender Year

In [164]:
_17 = Historical().get("2017-12-31")
_18 = Historical().get("2018-12-31")
_19 = Historical().get("2019-12-31")
_20 = Historical().get("2020-12-31")

responses = [_17.body['rates']
             ,_18.body['rates']
             ,_19.body['rates']
             ,_20.body['rates']]

RES = copy.deepcopy(responses)

sems = ['2017','2018','2019', '2020']

#includes housing, student services, tuition and manifest (last sem)
#excludes security deposits, insurance payments, etc.
sem_rates = [24450, 24950, 25950, 25950]

# Inflation Data!

In [285]:
#https://www.imf.org/external/datamapper/PCPIEPCH@WEO/WEOWORLD

#enter data
inflation_df =  pd.read_csv("data/imf_inflation.csv")

#store last row of euro data
euro_inflation_data = inflation_df.iloc[-1]
euro_inflation_data['Currency Code'] = "EUR"

inflation_df 

,Country,2017,2018,2019,2020
0,Afghanistan,3,0.8,2.8,5
1,Albania,1.8,1.8,1.1,1.2
2,Algeria,4.9,2.7,2.4,5.2
3,Angola,23.7,18.6,16.9,22.2
4,Antigua and Barbuda,2.4,1.7,1.5,0.7
...,...,...,...,...,...
190,West Bank and Gaza,0,0.3,1.3,-1.7
191,Yemen,47,14.3,6.2,45.4
192,Zambia,6.1,7.9,11.7,13
193,Zimbabwe,3.4,42.1,521.1,495


In [286]:
#find rows to drop in inflation dataset
inflation_data_to_drop = []
for index,country in enumerate(inflation_df.Country):
    if country not in list(nationality_df.Nationality):
        inflation_data_to_drop.append(index)
        
#drop rows with no Minervan nationality
inflation_df = inflation_df.drop(inflation_data_to_drop)
inflation_df = inflation_df.reset_index(drop=True)

#merge on nationality_df to find currencies of each country
inflation_df = inflation_df.merge(nationality_df, left_on='Country', right_on='Nationality').drop(columns = ['Nationality','Total Students'])
inflation_df

,Country,2017,2018,2019,2020,Currency Code
0,Albania,1.8,1.8,1.1,1.2,ALL
1,Algeria,4.9,2.7,2.4,5.2,DZD
2,Argentina,24.8,47.6,53.8,41.7,ARS
3,Armenia,2.6,1.8,0.7,1.5,AMD
4,Australia,2.1,1.7,1.8,0.6,AUD
...,...,...,...,...,...,...
72,Uruguay,6.6,8,8.8,9.5,UYU
73,Uzbekistan,18.8,14.3,15.2,12.1,UZS
74,Venezuela,862.6,130060.2,9585.5,6500,VEF
75,Vietnam,2.6,3,5.2,3.1,VND


In [287]:
more_inflation_data_to_drop = []
for index,currency in enumerate(inflation_df["Currency Code"]):
    if currency == "EUR":
        more_inflation_data_to_drop.append(index)
        
#drop Euro zone country respective inflation data
inflation_df = inflation_df.drop(more_inflation_data_to_drop)
inflation_df = inflation_df.reset_index(drop=True)

#drop venezuela because that inflation rate wow.
inflation_df = inflation_df.drop([62])

#include EURO
inflation_df = inflation_df.append(euro_inflation_data).drop(columns = ['Country'])

#convert to floats
inflation_df[['2017','2018','2019','2020']] = inflation_df[['2017','2018','2019','2020']].astype('float')

#sort alphabetic currency code
inflation_df = inflation_df.sort_values('Currency Code', ascending=True)

#reset index
inflation_df = inflation_df.reset_index(drop=True)


inflation_df.index = inflation_df['Currency Code']


inflation_df

,2017,2018,2019,2020,Currency Code
Currency Code,,,,,
AED,2.00000,3.10000,-1.90000,-1.50000,AED
ALL,1.80000,1.80000,1.10000,1.20000,ALL
AMD,2.60000,1.80000,0.70000,1.50000,AMD
ARS,24.80000,47.60000,53.80000,41.70000,ARS
AUD,2.10000,1.70000,1.80000,0.60000,AUD
...,...,...,...,...,...
UYU,6.60000,8.00000,8.80000,9.50000,UYU
UZS,18.80000,14.30000,15.20000,12.10000,UZS
VND,2.60000,3.00000,5.20000,3.10000,VND


# Convert to 2017 USD

In [316]:
for count,sem in enumerate(RES):

    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)

    #get rates for each currency        
    df[sems[count] +"_Rates"] = sem.values()
    
    #usd value in 2017
    #multiply exchange rate of that year with tuition, convert to 
    df[sems[count]] = [ ((rate * sem_rates[count])/ list(RES[0].values())[index]) for index,rate in enumerate(sem.values()) ]
    
    #df[sems[count] +"_Difference"] = [ round(((rate * sem_rates[count])/ list(RES[0].values())[index]  - sem_rates[0])) for index,rate in enumerate(sem.values()) ]

df

,CurrencyName,Students,2017_Rates,2017,2018_Rates,2018,2019_Rates,2019,2020_Rates,2020
AED,United Arab Emirates Dirham,1,3.67287,24450.00000,3.67278,24949.40221,3.67320,25952.35275,3.67301,25950.99621
ALL,Albanian Lek,1,110.81531,24450.00000,107.94662,24304.11753,108.56339,25422.66113,100.74137,23590.95124
AMD,Armenian Dram,3,483.25497,24450.00000,485.46483,25064.09320,478.89154,25715.69112,522.91000,28079.41025
ARS,Argentine Peso,3,19.15400,24450.00000,37.61500,48997.29821,59.84070,81072.68273,84.07466,113905.05383
AUD,Australian Dollar,3,1.28060,24450.00000,1.42048,27675.23126,1.42552,28886.58922,1.29885,26319.79662
...,...,...,...,...,...,...,...,...,...,...
UYU,Uruguayan Peso,2,28.79468,24450.00000,32.40837,28081.18745,37.21296,33536.61976,42.23501,38062.53660
UZS,Uzbekistan Som,1,8108.25000,24450.00000,8345.89174,25681.25043,9505.63945,30422.26666,10474.79528,33523.99561
VND,Vietnamese Dong,35,22701.41115,24450.00000,23116.20792,25405.88265,23179.64399,26496.66832,23192.97046,26511.90182
ZAR,South African Rand,4,12.37860,24450.00000,14.36125,28946.19200,14.01090,29371.90004,14.66265,30738.20312


In [317]:
explore = df.loc[:, df.columns.intersection([ sem for sem in sems ])]
#explore = explore.drop(index = "VEF")

#bar chart data
explore.columns = sems

explore['Currency'] = explore.index
first_col = explore.pop('Currency')
exlore =explore.insert(0, 'Currency', first_col)
explore["FullCurrency"] = df["CurrencyName"]


#explore = explore.drop(columns = 'total')
explore[58:]

,Currency,2017,2018,2019,2020,FullCurrency
UGX,UGX,24450.00000,25474.87580,26158.29402,26093.69600,Ugandan Shilling
USD,USD,24450.00000,24950.00000,25950.00000,25950.00000,United States Dollar
UYU,UYU,24450.00000,28081.18745,33536.61976,38062.53660,Uruguayan Peso
UZS,UZS,24450.00000,25681.25043,30422.26666,33523.99561,Uzbekistan Som
VND,VND,24450.00000,25405.88265,26496.66832,26511.90182,Vietnamese Dong
ZAR,ZAR,24450.00000,28946.19200,29371.90004,30738.20312,South African Rand
ZWL,ZWL,24450.00000,24950.00000,25921.42123,25921.42115,Zimbabwean Dollar


# Adjust For Inflation

In [318]:
'''how to adjust for inflation with inflation rates
inflation rate = cpi x year - cpi base year

'''
cpi17 = 100
cpi18 = ((inflation_df['2018']*cpi17/100)+cpi17)
cpi19 = ((inflation_df['2019']*cpi18/100)+cpi18)
cpi20 = ((inflation_df['2020']*cpi19/100)+cpi19)

explore['2017'] = explore['2017'] / cpi17 * cpi17
explore['2018'] = explore['2018'] / cpi18 * cpi17
explore['2019'] =  explore['2019'] / cpi19 * cpi17
explore['2020'] = explore['2020'] / cpi20 * cpi17

In [319]:
#line chart data

explore['Currency'] =  df["CurrencyName"]
explore = explore.drop(columns = 'FullCurrency')
explore = explore.sort_values('Currency', ascending=True)
explore = explore.drop(index = "ZWL")
explore = explore.round(decimals=0)

explore.to_csv("./ForexQuestIdyll/data/explore.csv",index = False)


import csv
with open("./ForexQuestIdyll/data/currencies.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(list(explore.Currency))

explore

,Currency,2017,2018,2019,2020
ALL,Albanian Lek,24450.00000,23874.00000,24701.00000,22650.00000
DZD,Algerian Dinar,24450.00000,25033.00000,25621.00000,27009.00000
ARS,Argentine Peso,24450.00000,33196.00000,35713.00000,35410.00000
AMD,Armenian Dram,24450.00000,24621.00000,25085.00000,26986.00000
AUD,Australian Dollar,24450.00000,27213.00000,27901.00000,25271.00000
...,...,...,...,...,...
AED,United Arab Emirates Dirham,24450.00000,24199.00000,25660.00000,26049.00000
USD,United States Dollar,24450.00000,24485.00000,24942.00000,24429.00000
UYU,Uruguayan Peso,24450.00000,26001.00000,28541.00000,29582.00000
UZS,Uzbekistan Som,24450.00000,22468.00000,23104.00000,22712.00000


In [235]:
explore = explore.sort_index(ascending=True)

#to observable to convert
sems_dates = ["December 31, 2017",
              "December 31, 2018",
              "December 31, 2019",
              "December 31, 2020",]

a = ['Currency'] + sems_dates  

explore.columns = a

explore['Students'] = list(df['Students'].drop(index = "ZWL"))

explore.to_csv("./ForexQuestIdyll/data/observable.csv", index=False)

explore

,Currency,"December 31, 2017","December 31, 2018","December 31, 2019","December 31, 2020",Students
AED,United Arab Emirates Dirham,24450.00000,24199.00000,25660.00000,26049.00000,1
ALL,Albanian Lek,24450.00000,23874.00000,24701.00000,22650.00000,1
AMD,Armenian Dram,24450.00000,24621.00000,25085.00000,26986.00000,3
ARS,Argentine Peso,24450.00000,33196.00000,35713.00000,35410.00000,3
AUD,Australian Dollar,24450.00000,27213.00000,27901.00000,25271.00000,3
...,...,...,...,...,...,...
USD,United States Dollar,24450.00000,24485.00000,24942.00000,24429.00000,76
UYU,Uruguayan Peso,24450.00000,26001.00000,28541.00000,29582.00000,2
UZS,Uzbekistan Som,24450.00000,22468.00000,23104.00000,22712.00000,1
VND,Vietnamese Dong,24450.00000,24666.00000,24453.00000,23732.00000,35


In [238]:
explore['total'] = explore.sum(axis=1)
explore = explore.drop(columns = ["Students"])
explore.to_csv("./ForexQuestIdyll/data/bar_data.csv")

In [239]:
explore

,Currency,"December 31, 2017","December 31, 2018","December 31, 2019","December 31, 2020",total
AED,United Arab Emirates Dirham,24450.00000,24199.00000,25660.00000,26049.00000,100359.00000
ALL,Albanian Lek,24450.00000,23874.00000,24701.00000,22650.00000,95676.00000
AMD,Armenian Dram,24450.00000,24621.00000,25085.00000,26986.00000,101145.00000
ARS,Argentine Peso,24450.00000,33196.00000,35713.00000,35410.00000,128772.00000
AUD,Australian Dollar,24450.00000,27213.00000,27901.00000,25271.00000,104838.00000
...,...,...,...,...,...,...
USD,United States Dollar,24450.00000,24485.00000,24942.00000,24429.00000,98382.00000
UYU,Uruguayan Peso,24450.00000,26001.00000,28541.00000,29582.00000,108576.00000
UZS,Uzbekistan Som,24450.00000,22468.00000,23104.00000,22712.00000,92735.00000
VND,Vietnamese Dong,24450.00000,24666.00000,24453.00000,23732.00000,97336.00000


# Short Term Volatility

## Illustrate Fin Aid Problem

In [133]:
'''
Packages Delivered to Students: Historical().get("2020-03-16")
Fall Term Bill: Historical().get("2020-07-15")
Spring Term Bill: Historical().get("2020-11-30")
'''


package = Historical().get("2020-03-16")
fall = Historical().get("2020-07-15")
spring = Historical().get("2020-11-30")

short = [package.body['rates']
             ,fall.body['rates']
             ,spring.body['rates']]


In [140]:
#export_fin_aid["Fin Aid Package Release"] = 
def percentages(df, col_name, initial):
    b = []
    for count, i in enumerate(list(df[col_name])):
        new = 100*(i-df[initial][count])/df[ initial][count]
        b.append(new)
    df[col_name] = b
    
fin_aid_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])

#fin_dates = ["2020-03-16","2020-07-15","2020-11-30"]
fin_dates = ["Fin Aid Package Release","Fall20 Bill Due","Spring21 Bill Due"]

for count,sem in enumerate(short):
    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)
            
    fin_aid_df[fin_dates[count]] = sem.values()
    
export_fin_aid = fin_aid_df


export_fin_aid.index = export_fin_aid['CurrencyName']
export_fin_aid = export_fin_aid.drop(columns = ["CurrencyName"])


export_fin_aid[:10]


,Fin Aid Package Release,Fall20 Bill Due,Spring21 Bill Due
CurrencyName,,,
United Arab Emirates Dirham,3.67330,3.67320,3.67320
Albanian Lek,109.40953,108.72440,103.68506
Armenian Dram,487.46314,481.61623,481.61623
Argentine Peso,62.96456,71.29047,81.30862
Australian Dollar,1.63877,1.42691,1.35885
Azerbaijani Manat,1.70250,1.70250,1.70250
Bangladeshi Taka,84.88204,84.77021,84.82090
Bulgarian Lev,1.75046,1.71325,1.63875
Brazilian Real,5.00090,5.37017,5.33260


In [141]:
percentages(export_fin_aid, "Fall20 Bill Due", "Fin Aid Package Release")
percentages(export_fin_aid, "Spring21 Bill Due", "Fin Aid Package Release")
percentages(export_fin_aid, "Fin Aid Package Release", "Fin Aid Package Release")

export_fin_aid = export_fin_aid
export_fin_aid = export_fin_aid.round(decimals=5)
export_fin_aid["Students"] = list(student_df['Total Students'])

hyp = []
hyp2 = []
for i in export_fin_aid['Fall20 Bill Due']:
    hyp.append((10000 * i/100) + 10000)
for i in export_fin_aid['Spring21 Bill Due']:
    hyp2.append((10000 * i/100) + 10000)
    
export_fin_aid["Hypothetical"] = hyp
export_fin_aid["Hypothetical2"] = hyp2

export_fin_aid = export_fin_aid.round(decimals=2)


export_fin_aid.to_csv("./ForexQuestIdyll/data/fin_aid_process_data.csv")

In [142]:
export_fin_aid

,Fin Aid Package Release,Fall20 Bill Due,Spring21 Bill Due,Students,Hypothetical,Hypothetical2
CurrencyName,,,,,,
United Arab Emirates Dirham,0.00000,-0.00000,-0.00000,1,9999.73000,9999.73000
Albanian Lek,0.00000,-0.63000,-5.23000,1,9937.38000,9476.78000
Armenian Dram,0.00000,-1.20000,-1.20000,3,9880.05000,9880.05000
Argentine Peso,0.00000,13.22000,29.13000,3,11322.32000,12913.40000
Australian Dollar,0.00000,-12.93000,-17.08000,3,8707.21000,8291.90000
...,...,...,...,...,...,...
Uruguayan Peso,0.00000,0.90000,-1.83000,2,10089.81000,9816.72000
Uzbekistan Som,0.00000,7.38000,9.29000,1,10738.02000,10929.10000
Vietnamese Dong,0.00000,-0.18000,-0.15000,35,9981.96000,9985.31000


## installments?

In [245]:
'''
Install1: Historical().get("2020-07-15")
Install2: Historical().get("2020-09-15")
Install3: Historical().get("2020-11-15")
'''


install1 = Historical().get("2020-07-15")
install2= Historical().get("2020-09-15")
install3= Historical().get("2020-11-15")

short = [install1.body['rates']
        , install2.body['rates']
        , install3.body['rates']]


In [258]:
install_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])

#fin_dates = ["2020-03-16","2020-07-15","2020-11-30"]
install_dates = ["July 15th, 2020 (1st Installment)","Sep 15th, 2020 (2nd Installment)","Nov 15th, 2020 (3rd Installment)"]

for count,sem in enumerate(short):
    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)
            
    install_df[install_dates[count]] = sem.values()
    
export_install = install_df


export_install.index = export_install['CurrencyName']
export_install = export_install.drop(columns = ["CurrencyName"])

percentages(export_install, "Sep 15th, 2020 (2nd Installment)", "July 15th, 2020 (1st Installment)")
percentages(export_install, "Nov 15th, 2020 (3rd Installment)", "July 15th, 2020 (1st Installment)")
percentages(export_install, "July 15th, 2020 (1st Installment)", "July 15th, 2020 (1st Installment)")

hyp = []
hyp2 = []
for i in export_install['Sep 15th, 2020 (2nd Installment)']:
    hyp.append((10000 * i/100))
for i in export_install['Nov 15th, 2020 (3rd Installment)']:
    hyp2.append((10000 * i/100))
    
export_install["Sep 15th, 2020 (2nd Installment)"] = hyp
export_install["Nov 15th, 2020 (3rd Installment)"] = hyp2
export_install["July 15th, 2020 (1st Installment)"] = 0
export_install["4% Service Fee"] = 1200
export_install["total"] = 1200 + export_install["Sep 15th, 2020 (2nd Installment)"] + export_install["Nov 15th, 2020 (3rd Installment)"]


export_install = export_install.round(decimals=2)


export_install.to_csv("./ForexQuestIdyll/data/installments_data.csv")


In [259]:
export_install

,"July 15th, 2020 (1st Installment)","Sep 15th, 2020 (2nd Installment)","Nov 15th, 2020 (3rd Installment)",4% Service Fee,total
CurrencyName,,,,,
United Arab Emirates Dirham,0,0.00000,-0.63000,1200,1199.37000
Albanian Lek,0,-391.98000,-368.02000,1200,440.00000
Armenian Dram,0,0.00000,0.00000,1200,1200.00000
Argentine Peso,0,540.69000,1174.64000,1200,2915.33000
Australian Dollar,0,-395.08000,-386.41000,1200,418.51000
...,...,...,...,...,...
Uruguayan Peso,0,-316.09000,-222.75000,1200,661.16000
Uzbekistan Som,0,61.43000,168.45000,1200,1429.88000
Vietnamese Dong,0,22.75000,-47.47000,1200,1175.28000


## Get Quarterly Rates

In [569]:
quarterly_rates = []
quarterly_dates =[]

for year in tqdm([2017,2018,2019,2020]):
    for month in ["01","03","06","09"]:
        date = f"{year}-{month}-01"
        quarterly_dates.append(date)
        quarterly_rates.append(Historical().get(date).body['rates'])
        
        

100%|██████████| 4/4 [00:04<00:00,  1.23s/it]


In [571]:
quarterly_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
for count,sem in enumerate(quarterly_rates):
    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)
            
    quarterly_df[quarterly_dates[count]] = sem.values()
    
quarterly_df = quarterly_df.drop(columns = ["CurrencyName"])


In [572]:
quarterly_df

,2017-01-01,2017-03-01,2017-06-01,2017-09-01,2018-01-01,2018-03-01,2018-06-01,2018-09-01,2019-01-01,2019-03-01,2019-06-01,2019-09-01,2020-01-01,2020-03-01,2020-06-01,2020-09-01
AED,3.67290,3.67261,3.67290,3.67302,3.67278,3.67310,3.67316,3.67318,3.67311,3.67300,3.67274,3.67299,3.67320,3.67303,3.67320,3.67320
ALL,128.29838,128.00312,119.58839,112.65000,110.66180,108.12000,107.50000,108.99000,107.72677,110.30000,109.75000,110.75773,109.15356,111.70000,111.73701,103.59047
AMD,483.72750,485.95623,483.13877,479.97899,483.16490,480.78038,483.02000,482.91541,484.06811,490.04460,480.42521,476.37149,481.45533,478.38209,483.42300,481.61623
ARS,15.86875,15.45000,16.05100,17.21000,18.59141,20.14750,24.94800,36.84200,37.65150,39.82500,44.69400,59.50300,59.92753,61.99251,68.57827,74.24998
AUD,1.38620,1.30563,1.35494,1.25485,1.28221,1.28790,1.32116,1.39005,1.41881,1.41243,1.44050,1.48680,1.42450,1.54568,1.47407,1.35555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UYU,29.24560,28.19378,28.29338,28.97421,28.82706,28.35343,31.21418,32.21382,32.36172,32.63410,35.24947,36.63912,37.33226,38.96242,43.30512,42.60436
UZS,3195.62661,3340.75000,3851.20000,4198.90000,8101.40000,8186.98333,7990.00000,7830.83443,8318.96846,8400.00000,8490.00000,9385.69491,9505.63945,9526.00000,10151.04352,10259.04897
VND,22792.50000,22782.96667,22696.90565,22743.62347,22700.88386,22747.11003,22797.27535,23114.33767,23177.00678,23216.86756,23303.00905,23285.59634,23191.33305,23120.15961,23224.12301,23300.46417
ZAR,13.78381,13.02427,12.89545,12.94035,12.40178,11.86436,12.69016,14.68696,14.39440,14.22819,14.58455,15.23625,14.00950,15.80901,17.38700,16.63200


## Get Monthly Rates

In [642]:
monthly_rates = []
monthly_dates =[]

for year in tqdm([2017,2018,2019,2020]):
    for month in range(1,13):
        if month < 10:
            month = "0"+str(month)
        date = f"{year}-{month}-01"
        monthly_dates.append(date)
        monthly_rates.append(Historical().get(date).body['rates'])
        
        

100%|██████████| 4/4 [00:16<00:00,  4.08s/it]


In [658]:
monthly_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
for count,sem in enumerate(monthly_rates):
    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)
            
    monthly_df[monthly_dates[count]] = sem.values()

#monthly_df = monthly_df.drop(columns = ["CurrencyName"])


### export rates for visual

In [659]:
export_monthly = monthly_df
export_monthly.index = export_monthly['CurrencyName']
export_monthly = export_monthly.drop(columns = ["CurrencyName"])
export_monthly.T.to_csv("./ForexQuestIdyll/data/monthly_data.csv")


## Get Weekly Rates

In [562]:
weekly_rates = []
weekly_dates =[]

for year in tqdm([2017,2018,2019,2020]):
    for month in range(1,13):
        for week in ["01","07","14","21"]:
            if month < 10:
                month_str = "0"+ str(month)
                date = f"{year}-{month_str}-{week}"
                weekly_dates.append(date)
                weekly_rates.append(Historical().get(date).body['rates'])
            else:
                date = f"{year}-{month}-{week}"
                weekly_dates.append(date)
                weekly_rates.append(Historical().get(date).body['rates'])
        

100%|██████████| 4/4 [01:01<00:00, 15.26s/it]


In [623]:
weekly_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
for count,sem in enumerate(weekly_rates):
    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)
            
    weekly_df[weekly_dates[count]] = sem.values()

weekly_df = weekly_df.drop(columns = ["CurrencyName"])
weekly_df

,2017-01-01,2017-01-07,2017-01-14,2017-01-21,2017-02-01,2017-02-07,2017-02-14,2017-02-21,2017-03-01,2017-03-07,2017-03-14,2017-03-21,2017-04-01,2017-04-07,2017-04-14,2017-04-21,2017-05-01,2017-05-07,2017-05-14,2017-05-21,2017-06-01,2017-06-07,2017-06-14,2017-06-21,2017-07-01,2017-07-07,2017-07-14,2017-07-21,2017-08-01,2017-08-07,2017-08-14,2017-08-21,2017-09-01,2017-09-07,2017-09-14,2017-09-21,2017-10-01,2017-10-07,2017-10-14,2017-10-21,2017-11-01,2017-11-07,2017-11-14,2017-11-21,2017-12-01,2017-12-07,2017-12-14,2017-12-21,2018-01-01,2018-01-07,2018-01-14,2018-01-21,2018-02-01,2018-02-07,2018-02-14,2018-02-21,2018-03-01,2018-03-07,2018-03-14,2018-03-21,2018-04-01,2018-04-07,2018-04-14,2018-04-21,2018-05-01,2018-05-07,2018-05-14,2018-05-21,2018-06-01,2018-06-07,2018-06-14,2018-06-21,2018-07-01,2018-07-07,2018-07-14,2018-07-21,2018-08-01,2018-08-07,2018-08-14,2018-08-21,2018-09-01,2018-09-07,2018-09-14,2018-09-21,2018-10-01,2018-10-07,2018-10-14,2018-10-21,2018-11-01,2018-11-07,2018-11-14,2018-11-21,2018-12-01,2018-12-07,2018-12-14,2018-12-21,2019-01-01,2019-01-07,2019-01-14,2019-01-21,2019-02-01,2019-02-07,2019-02-14,2019-02-21,2019-03-01,2019-03-07,2019-03-14,2019-03-21,2019-04-01,2019-04-07,2019-04-14,2019-04-21,2019-05-01,2019-05-07,2019-05-14,2019-05-21,2019-06-01,2019-06-07,2019-06-14,2019-06-21,2019-07-01,2019-07-07,2019-07-14,2019-07-21,2019-08-01,2019-08-07,2019-08-14,2019-08-21,2019-09-01,2019-09-07,2019-09-14,2019-09-21,2019-10-01,2019-10-07,2019-10-14,2019-10-21,2019-11-01,2019-11-07,2019-11-14,2019-11-21,2019-12-01,2019-12-07,2019-12-14,2019-12-21,2020-01-01,2020-01-07,2020-01-14,2020-01-21,2020-02-01,2020-02-07,2020-02-14,2020-02-21,2020-03-01,2020-03-07,2020-03-14,2020-03-21,2020-04-01,2020-04-07,2020-04-14,2020-04-21,2020-05-01,2020-05-07,2020-05-14,2020-05-21,2020-06-01,2020-06-07,2020-06-14,2020-06-21,2020-07-01,2020-07-07,2020-07-14,2020-07-21,2020-08-01,2020-08-07,2020-08-14,2020-08-21,2020-09-01,2020-09-07,2020-09-14,2020-09-21,2020-10-01,2020-10-07,2020-10-14,2020-10-21,2020-11-01,2020-11-07,2020-11-14,2020-11-21,2020-12-01,2020-12-07,2020-12-14,2020-12-21
AED,3.67290,3.67302,3.67302,3.67302,3.67278,3.67310,3.67301,3.67301,3.67261,3.67270,3.67301,3.67301,3.67290,3.67290,3.67290,3.67310,3.67310,3.67290,3.67294,3.67302,3.67290,3.67310,3.67301,3.67301,3.67301,3.67301,3.67302,3.67302,3.67310,3.67310,3.67309,3.67310,3.67302,3.67301,3.67301,3.67310,3.67297,3.67290,3.67310,3.67290,3.67290,3.67310,3.67270,3.67278,3.67310,3.67290,3.67310,3.67238,3.67278,3.67293,3.67293,3.67294,3.67310,3.67310,3.67310,3.67310,3.67310,3.67310,3.67301,3.67310,3.67290,3.67301,3.67302,3.67301,3.67302,3.67318,3.67301,3.67318,3.67316,3.67318,3.67318,3.67318,3.67301,3.67318,3.67318,3.67318,3.67318,3.67318,3.67318,3.67318,3.67318,3.67330,3.67301,3.67301,3.67318,3.67303,3.67298,3.67296,3.67318,3.67318,3.67310,3.67310,3.67318,3.67301,3.67301,3.67301,3.67311,3.67318,3.67318,3.67318,3.67301,3.67318,3.67310,3.67313,3.67300,3.67318,3.67318,3.67318,3.67318,3.67304,3.67303,3.67302,3.67301,3.67318,3.67318,3.67318,3.67274,3.67301,3.67318,3.67318,3.67318,3.67303,3.67292,3.67296,3.67318,3.67318,3.67318,3.67318,3.67299,3.67302,3.67296,3.67302,3.67318,3.67318,3.67320,3.67300,3.67286,3.67320,3.67320,3.67320,3.67294,3.67300,3.67300,3.67320,3.67320,3.67320,3.67320,3.67320,3.67310,3.67300,3.67300,3.67320,3.67303,3.67300,3.67300,3.67300,3.67320,3.67320,3.67320,3.67320,3.67305,3.67320,3.67310,3.67320,3.67320,3.67299,3.67297,3.67298,3.67302,3.67320,3.67320,3.67320,3.67295,3.67300,3.67315,3.67294,3.67320,3.67320,3.67320,3.67320,3.67320,3.67320,3.67320,3.67299,3.67297,3.67300,3.67300,3.67300,3.67320,3.67320,3.67320,3.67310
ALL,128.29838,128.57500,128.65000,127.91824,126.86598,127.65500,128.13612,128.17112,128.00312,127.97500,127.10000,125.27612,127.67615,128.00000,127.71500,125.76444,123.87948,122.98739,120.50524,120.18333,119.58839,118.80775,119.05000,118.80775,115.98224,116.15000,115.37500,113.67879,112.45320,112.20447,112.51104,112.00884,112.65000,111.31500,111

## Calculate Volatility

In [611]:
quarterly_dev = []
for index, row in quarterly_df.iterrows():
    vol = []
    counter = 0
    while counter < len(row)-1:
        ret = np.log(row[counter+1]/row[counter])
        vol.append(ret)
        counter += 1
    quarterly_dev.append(np.std(vol)*100)

month_dev = []
for index, row in monthly_df.iterrows():
    vol = []
    counter = 0
    while counter < len(row)-1:
        ret = np.log(row[counter+1]/row[counter])
        vol.append(ret)
        counter += 1
    month_dev.append(np.std(vol)*100)
    
week_dev = []
for index, row in weekly_df.iterrows():
    vol = []
    counter = 0
    while counter < len(row)-1:
        ret = np.log(row[counter+1]/row[counter])
        vol.append(ret)
        counter += 1
    week_dev.append(np.std(vol)*100)

In [615]:
vol_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
vol_df["Weekly Volatility"] = week_dev
vol_df["Monthly Volatility"] = month_dev
vol_df["Quarterly Volatility"] = quarterly_dev


vol_df

,CurrencyName,Weekly Volatility,Monthly Volatility,Quarterly Volatility
AED,United Arab Emirates Dirham,0.00384,0.00411,0.00538
ALL,Albanian Lek,1.00560,2.05217,3.00088
AMD,Armenian Dram,0.47136,0.68662,0.85060
ARS,Argentine Peso,3.45144,12.95628,10.79820
AUD,Australian Dollar,1.51046,2.07003,4.77043
...,...,...,...,...
UYU,Uruguayan Peso,1.25673,3.45836,4.01310
UZS,Uzbekistan Som,4.76519,10.96315,16.12959
VND,Vietnamese Dong,0.41190,0.46067,0.42505
ZAR,South African Rand,2.23036,3.39691,6.65125


In [616]:
vol_df.to_csv("./ForexQuestIdyll/data/volalitity_data.csv")

## Time  / Percentage

In [629]:
weekly_df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
for count,sem in enumerate(weekly_rates):
    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)
            
    weekly_df[weekly_dates[count]] = sem.values()

weekly_df.index = weekly_df["CurrencyName"]
weekly_df = weekly_df.drop(columns = ["CurrencyName"])


        
    

In [637]:
weekly_percentage_df = weekly_df.T.div(weekly_df.T.iloc[0]/100)
weekly_percentage_df.to_csv("./ForexQuestIdyll/data/weekly_percentage_data.csv")


In [641]:
monthly_df

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2018-01-01,2018-06-01,2019-01-01,2019-06-01,2020-01-01
AED,3.67290,3.67278,3.67261,3.67290,3.67310,3.67278,3.67316,3.67311,3.67274,3.67320,3.67320
ALL,119.58839,126.86598,128.00312,127.67615,123.87948,110.66180,107.50000,107.72677,109.75000,109.15356,111.73701
AMD,483.13877,486.28500,485.95623,484.70288,485.99903,483.16490,483.02000,484.06811,480.42521,481.45533,483.42300
ARS,16.05100,15.79400,15.45000,15.37925,15.38350,18.59141,24.94800,37.65150,44.69400,59.92753,68.57827
AUD,1.35494,1.31869,1.30563,1.31053,1.32770,1.28221,1.32116,1.41881,1.44050,1.42450,1.47407
...,...,...,...,...,...,...,...,...,...,...,...
UYU,28.29338,28.14615,28.19378,28.65384,28.12650,28.82706,31.21418,32.36172,35.24947,37.33226,43.30512
UZS,3851.20000,3271.35995,3340.75000,3601.85000,3717.50000,8101.40000,7990.00000,8318.96846,8490.00000,9505.63945,10151.04352
VND,22696.90565,22586.81667,22782.96667,22774.34338,22760.33333,22700.88386,22797.27535,23177.00678,23303.00905,23191.33305,23224.12301
ZAR,12.89545,13.46447,13.02427,13.42341,13.39678,12.40178,12.69016,14.39440,14.58455,14.00950,17.38700


## Working Example - BRL 2020 to USD 2017

In [313]:
_20.body['rates']['BRL'] * 25950

134773.92

In [314]:
BRL_cpi17 = 100
BRL_cpi18 = (inflation_df['2018']['BRL']*BRL_cpi17/100)+BRL_cpi17
BRL_cpi19 = ((inflation_df['2019']['BRL']*BRL_cpi18/100)+BRL_cpi18)
BRL_cpi20 = ((inflation_df['2020']['BRL']*BRL_cpi19/100)+BRL_cpi19)

134773.92 / BRL_cpi20 * BRL_cpi17

122163.8254364608

In [315]:
122163.825/_17.body['rates']['BRL']

36890.59274042682